# CAPTAIN tutorial – conservation policy optimization

In CAPTAIN conservation policies are optimized through Reinforcement Learning (RL) using simulated natural systems. This involves three main steps:
1. Setting up the parameters of the simulated systems, including the number of spatial units, the size of the protection units, models of disturbance and climate change, and the budget
2. Defining the monitoring policy, namely, the frequency at which the agent monitors the system and the amount of information collected from the system
3. Choosing the policy objective (e.g., minimizing species loss, minimizing economic value loss, maximizing amount of protected area) and allocating a budget for conservation

The optimization is done within the constraints of a pre-defined budget.
Cost of protection is calculated per unit and based on a baseline cost plus additional cost in te presence of anthropogenic disturbance.

### Algorithm settings
Model training through RL is performed using the `train_model()` function in CAPTAIN.
There are a number of parameters that can be tweaked to adjust the settings of the optimization algorithm, depending on the available computing resources.
These include the number of time steps each system is set to evolve for (e.g. `steps=25`), the number of systems that will be run in parallel (this requires multiple CPUs, e.g. `batchSize=6` uses 6 threads) and the number of epochs run to optimize the parameters (e.g. `epochs=100`).
The policy is by default parameterized through a neural network with two hidden layers with number of nodes specified using the `n_nodes` argument.

### Monitoring and protection policy
The amount and type of information collected through monitoring of the system is defined by the `obsMode` parameter. The information is collected for each protection unit, which may contain more than one cell. The size of the protection units is user-defined, with e.g.,  `resolution = [5,2]` setting protection units of 5 x 2 cells. Note that currently the dimension of the protection units (by default set to 5 x 5) must be multiples of the dimension of the system.

0. Only monitor which unit is already protected
1. Full species monitoring: information about species abundance in each unit, including population sizes and their variation since previous monitoring
2. Citizen-science species monitoring: information about species presence and absence in each unit, ignoring population sizes
3. One-time full species monitoring: information about species abundance in each unit, including population sizes but without the temporal component of `obsmode=1`
4. Full value monitoring: similar to `obsmode=1` but with metrics weighted by species value
5. Area monitoring: information limited to the cost of protection units

The policy may involve recurrent monitoring at each step or a single monitoring event on the first step. The establishment of protected units can be carried out dynamically through time (one unit per time step until the budget allows) or spending all the budget in one step. These options are controlled by the `observePolicy` parameter:

0. One-time monitoring, dynamic protection
1. Recurrent monitoring, dynamic protection
2. Recurrent monitoring, protection in one step (all budget is spent in one step)

Before protection units start being selected the system can be monitored for a user-defined number of steps. This allows the agent to learn about spatio-temporal trends in species abundance and ranges. The number of steps after which the establishment of protected units begins is controlled through the argument `start_protecting`, which is by default set to 3. 

### Policy objective
The policy objective is controlled by the `rewardMode` argument with te following options:
0. minimize species loss
1. minimize economic value loss
2. maximize protected area
3. minimize loss of phylogenetic diversity

---

## Model training through reinforcement learning

The command used to train a model using **full recurrent monitoring** and aiming to **minimize biodiversity loss** can be written as follows:

In [ ]:
import os
os.chdir("path_to_captain-dev")
import captain as cn

path_to_sim_data = "data_dependencies/pickles" # path to pre-simulated datasets

# train a model using RL
cn.train_model(budget=0.11,
               steps=25,
               batchSize=1,
               epochs=10,
               n_nodes=[4,2],
               obsMode=1, # full species monitoring
               observePolicy=1, # recurrent monitoring, dynamic protection
               rewardMode=0, # objective: minimize species loss
               outfile = "full_recurrent_monitoring.log",
               wd = path_to_sim_data) # directory with pre-simulated systems

where the budget is defined as a maximum fraction of area that can be protected, e.g., `budget=0.1` means that up to 10% of the system's cells can be protected. However, since the cost of protection is a function of anthropogenic disturbance, the fraction of protected area will likely be lower.
The `train_model()` function uses simulated systems generated beforehand and stored in a directory indicated with the argument `wd` (pre-simulated datasets are available for [download here](https://zenodo.org/api/files/c4663248-5dc2-4932-9d43-ec0c490595ae/data_dependencies.zip?versionId=d29260c1-7449-4075-aa83-6e32e5f0db7e)).
Note that for the optimization to converge you will need to increase the number of epochs to a greater number (e.g. `epochs=1000`).

The bath size indicates the number of systems that are processed in each epoch.
Depending on the available resources (number of CPUs and amount of RAM), a larger batch size (e.g. `batchSize=6`) can also improve the optimization process. Note that setting a `batchSize` greater than one involves running all systems in parallel using the `ProcessPoolExecutor` [Python class](https://docs.python.org/3/library/concurrent.futures.html). 

The output of this function is a tab-separated log file (`outfile`) which provides an account of the reward and other summary statistics from each training epoch. The log file also includes the parameters of the neural network optimized in the process.
The log file can then be used to perform conservation planning on simulated and empirical datasets as shown in the next tutorials.

A model using **citizen-science monitoring** to minimize biodiversity loss can be optimized using:


In [ ]:
cn.train_model(budget=0.11,
               steps=25,
               batchSize=2,
               epochs=5,
               n_nodes=[4,2],
               obsMode=2, # citizen-science species monitoring
               observePolicy=1, # recurrent monitoring, dynamic protection
               rewardMode=0, # objective: minimize species loss
               outfile = "citizen-science_recurrent_monitoring.log",
               wd = path_to_sim_data) # directory with pre-simulated systems

while a model monitoring the system for the first few steps (in this case 5) before **all the budget is spent in one step** can be optimized using the following command:

In [ ]:
cn.train_model(budget=0.11,
               steps=25,
               batchSize=1,
               epochs=10,
               n_nodes=[4,2],
               obsMode=1, # full species monitoring
               observePolicy=2, # recurrent monitoring, at-once protection
               start_protecting=5, # system is monitored for 5 steps before placing protection units
               rewardMode=0, # objective: minimize species loss
               outfile = "full_monitoring_at-once_protection.log",
               wd = path_to_sim_data) # directory with pre-simulated systems

Training a model aiming to **minimize the loss of economic value** requires changing the monitoring strategy and the type of reward:


In [ ]:
cn.train_model(budget=0.11,
               steps=25,
               batchSize=1,
               epochs=10,
               n_nodes=[4,2],
               obsMode=4, # full species-value monitoring
               observePolicy=1, # recurrent monitoring, dynamic protection
               rewardMode=1, # objective: minimize value loss
               outfile = "full_monitoring_value.log",
               wd = path_to_sim_data) # directory with pre-simulated systems

Similarly, a model aiming to protect the largest amount of area (regardless of the number of species) is optimized using:


In [ ]:
cn.train_model(budget=0.11,
               steps=25,
               batchSize=1,
               epochs=10,
               n_nodes=[4,0],
               obsMode=5, # area-cost monitoring
               observePolicy=1, # recurrent monitoring, dynamic protection
               rewardMode=2, # objective: minimize value loss
               outfile = "full_monitoring_area.log",
               wd = path_to_sim_data) # directory with pre-simulated systems

In this case, since the amount of information needed for this model is limited, a simpler neural network with a single hidden layer can be used (here specified with `n_nodes=[4,0]`).